In [34]:
import requests
import json

url = "https://ollama-api.lab.groupe-genes.fr/api/generate"

payload = {
    "model": "gpt-oss:20b", # Remplacez par llama3.3:70b-instruct-q4_K_M si disponible
    "prompt": "Qu'est ce que le CREST ?",
    "stream": False,
    "options": {
        "num_predict": 1000, # Ajusté pour les résumés de 750 mots 
        "temperature": 0,    # CRUCIAL : force le greedy sampling 
        "seed": 42           # Recommandé pour la reproductibilité 
    }
}

headers = {
"Content-Type": "application/json"
}

response = requests.post(url, headers=headers, data=json.dumps(payload), verify=False)
print(response.text)






<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>



c:\Users\Thomas LUCEREAU\Documents\RA_Lucereau\RA_International_Trade\Research\Code\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ollama-api.lab.groupe-genes.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [12]:
import Prompts_clayton
import pandas as pd

prompt_system = Prompts_clayton.prompt_1

prompt_user = pd.read_csv("sp500_transcripts_history.csv").iloc[0]['content']

print(prompt_user)

Apple
(
AAPL
+0.28%
)
Q1 2025 Earnings Call
Jan 30, 2025
,
5:00 p.m. ET
Contents:
Prepared Remarks
Questions and Answers
Call Participants
Prepared Remarks:
Suhasini Chandramouli
--
Director, Investor Relations
Good afternoon, and welcome to the Apple Q1 fiscal year 2025 earnings conference call. My name is Suhasini Chandramouli, Director of investor relations. Today's call is being recorded. Speaking first today are Apple CEO, Tim Cook; and he'll be followed by CFO, Kevan Parekh.
After that, we'll open the call to questions from analysts. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, operating expenses, other income and expense, taxes, capital allocation, and future business outlook, including the potential impact of macroeconomic conditions on the company's business and results of operations. These statements involve risks and uncertaint

In [13]:
import os
import httpx
from openai import OpenAI

# 1. L'URL de base pour OpenAI doit pointer vers le domaine du labo
# Note: On garde le suffixe /v1 pour la compatibilité OpenAI
GENES_BASE_URL = "https://ollama-api.lab.groupe-genes.fr/v1"

client = OpenAI(
    base_url=GENES_BASE_URL,
    api_key="ollama", # Requis par la lib, mais ignoré par le serveur
    # Important: verify=False car votre serveur labo semble nécessiter de bypasser SSL
    http_client=httpx.Client(verify=False)
)

try:
    response = client.chat.completions.create(
        model="gpt-oss:20b",
        messages=[{"role": "system", "content": prompt_system},{"role": "user", "content": "Here is the transcript of a company earnings call :" + prompt_user }],
        temperature=0,
        seed=42
    )
    print("Réponse du serveur GENES :")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Erreur avec le client OpenAI : {e}")

Erreur avec le client OpenAI : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>


In [14]:
import httpx
import json
import re
from openai import OpenAI

# Configuration
GENES_BASE_URL = "https://ollama-api.lab.groupe-genes.fr/v1"
client = OpenAI(
    base_url=GENES_BASE_URL,
    api_key="ollama",
    http_client=httpx.Client(verify=False)
)

# 1. Découpage du transcript en 5 parties
def split_text(text, n):
    length = len(text)
    return [text[i*length // n: (i+1)*length // n] for i in range(n)]

def analyze_transcript(prompt_user, prompt_system):

    parts = split_text(prompt_user, 5)

    # Initialisation du dictionnaire de résultats (tous à 0)
    final_results = {
        "tariffs_any": 0, "sanctions_any": 0, "export_controls_any": 0,
        "boycotts_any": 0, "investment_screening_any": 0,
        "geo_subsidies_any": 0, "geoeconomic_any": 0
    }
    full_summary = ""

    # 2. Boucle sur les 5 parties
    for i, part in enumerate(parts):
        print(f"Analyse de la partie {i+1}/5...")
        try:
            response = client.chat.completions.create(
                model="gpt-oss:20b",
                messages=[
                    {"role": "system", "content": prompt_system},
                    {"role": "user", "content": f"PART {i+1} OF 5:\n\n{part}"}
                ],
                temperature=0,
                seed=42
            )
            content = response.choices[0].message.content
            
            # Extraction du JSON entre les tags <JSON>
            json_match = re.search(r"<JSON>(.*?)</JSON>", content, re.DOTALL)
            if json_match:
                data = json.loads(json_match.group(1).strip())
                # Mise à jour des flags (Logique OR : si on trouve un 1, on garde 1)
                for key in final_results:
                    if data.get(key) == 1:
                        final_results[key] = 1
            
            # Extraction du résumé entre les tags <SUMMARY> (pour cumul)
            summary_match = re.search(r"<SUMMARY>(.*?)</SUMMARY>", content, re.DOTALL)
            if summary_match:
                summary_text = summary_match.group(1).strip()
                if summary_text.lower() != "not affected.":
                    full_summary += f"\n--- Part {i+1} ---\n{summary_text}\n"

        except Exception as e:
            print(f"Erreur sur la partie {i+1} : {e}")
            
    return final_results, full_summary

final_results, full_summary = analyze_transcript(prompt_user, prompt_system)

# 3. Affichage final
print("\n" + "="*20)
print("RÉSULTATS FINAUX (AGRÉGÉS)")
print("="*20)
print(json.dumps(final_results, indent=4))
print("\nSYNTHÈSE DES IMPACTS :")
print(full_summary if full_summary else "Not affected.")

Analyse de la partie 1/5...
Erreur sur la partie 1 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 2/5...
Erreur sur la partie 2 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 3/5...
Erreur sur la partie 3 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 4/5...
Erreur sur la partie 4 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 5/5...
Erreur sur la partie 5 : <ht

In [15]:
# selectionner les 10 premiers resultats du csv pour les tester

data_10 = pd.read_csv("sp500_transcripts_history.csv").iloc[:10]

for index, row in data_10.iterrows():
    print(f"\n\n=== Analyse du transcript {row['company']} | Année {row['year']} | Q{row['quarter']} ===")
    final_results, full_summary = analyze_transcript(row['content'], prompt_system)
    print("\n" + "="*20)
    print("RÉSULTATS FINAUX (AGRÉGÉS)")
    print("="*20)
    print(json.dumps(final_results, indent=4))
    print("\nSYNTHÈSE DES IMPACTS :")
    print(full_summary if full_summary else "Not affected.")



=== Analyse du transcript Apple Inc. (AAPL) | Année 2025 | QQ1 ===
Analyse de la partie 1/5...
Erreur sur la partie 1 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 2/5...
Erreur sur la partie 2 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 3/5...
Erreur sur la partie 3 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 4/5...
Erreur sur la partie 4 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</b

In [16]:

# Sélectionner les 10 premiers résultats
data = pd.read_csv("sp500_transcripts_history_full.csv")

# Liste pour accumuler les données
all_results = []

for index, row in data.iterrows():
    print(f"\n\n=== Analyse du transcript {row['company']} | Année {row['year']} | {row['quarter']} ===")
    
    # Appel de ta fonction (qui gère déjà le découpage en 5 parties et l'agrégation)
    final_results, full_summary = analyze_transcript(row['content'], prompt_system)
    
    # On prépare le dictionnaire pour cette ligne
    entry = {
        'company': row['company'],
        'year': row['year'],
        'quarter': row['quarter'],
        'explication': full_summary if full_summary else "Not affected."
    }
    
    # On ajoute dynamiquement les colonnes du JSON (tariffs_any, sanctions_any, etc.)
    entry.update(final_results)
    
    # On ajoute la ligne à notre liste
    all_results.append(entry)

    # Print de contrôle (optionnel)
    print(f"Statut : {entry.get('geoeconomic_any', 0)} flag(s) détecté(s)")

# Création du DataFrame final
df_analyse = pd.DataFrame(all_results)


df_analyse.to_csv("results_extraction.csv", index=False)



=== Analyse du transcript Apple Inc. (AAPL) | Année 2025 | Q1 ===
Analyse de la partie 1/5...
Erreur sur la partie 1 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 2/5...
Erreur sur la partie 2 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 3/5...
Erreur sur la partie 3 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 4/5...
Erreur sur la partie 4 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</bo

KeyboardInterrupt: 

In [17]:
df_analyse.head(10)

NameError: name 'df_analyse' is not defined

In [37]:
import requests 
import re
import json

data = pd.read_csv("sp500_transcripts_history_full.csv")

transcript_to_test = data.iloc[6]['content']

def split_text_smart(text, n):
    if not isinstance(text, str) or len(text) == 0:
        return [""] * n
    parts = []
    current_idx = 0
    target_len = len(text) // n
    for i in range(n - 1):
        search_start = current_idx + target_len
        next_separator = text.find('.', search_start)
        if next_separator == -1:
            end_idx = len(text)
        else:
            end_idx = next_separator
        parts.append(text[current_idx:end_idx])
        current_idx = end_idx
    parts.append(text[current_idx:])
    return parts

def analyze_transcript(prompt_user, prompt_system):
    parts = split_text_smart(prompt_user, 5)

    final_results = {
        "tariffs_any": 0, "sanctions_any": 0, "export_controls_any": 0,
        "boycotts_any": 0, "investment_screening_any": 0,
        "geo_subsidies_any": 0, "geoeconomic_any": 0
    }
    full_summary = ""

    for i, part in enumerate(parts):
        if not part.strip(): continue
        print(f"Analyse de la partie {i+1}/5...")
        try:
            response = client.chat.completions.create(
                model="gpt-oss:20b",
                messages=[
                    {"role": "system", "content": prompt_system},
                    {"role": "user", "content": f"PART {i+1} OF 5:\n\n{part}"}
                ],
                temperature=0,
                seed=42
            )
            content = response.choices[0].message.content
            
            # 1. Extraction et analyse du JSON
            current_part_has_policy = False
            json_match = re.search(r"<JSON>(.*?)</JSON>", content, re.DOTALL)
            if json_match:
                try:
                    data = json.loads(json_match.group(1).strip())
                    for key in final_results:
                        if data.get(key) == 1:
                            final_results[key] = 1
                            current_part_has_policy = True # On a trouvé une catégorie active !
                except: pass
            
            # 2. Extraction du résumé UNIQUEMENT si une catégorie a été détectée
            summary_match = re.search(r"<SUMMARY>(.*?)</SUMMARY>", content, re.DOTALL)
            if summary_match and current_part_has_policy:
                summary_text = summary_match.group(1).strip()
                print(summary_text)
                # On ajoute au résumé global
                if summary_text.lower() != "not affected.":
                    full_summary += f"\n--- Part {i+1} ---\n{summary_text}\n"

        except Exception as e:
            print(f"Erreur sur la partie {i+1} : {e}")
            
    return final_results, full_summary

final_results, full_summary = analyze_transcript(transcript_to_test, prompt_system)

print(full_summary)
print(json.dumps(final_results, indent=4))


Analyse de la partie 1/5...
Erreur sur la partie 1 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 2/5...
Erreur sur la partie 2 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 3/5...
Erreur sur la partie 3 : <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>
Analyse de la partie 4/5...


KeyboardInterrupt: 

In [38]:
import openpyxl

results = pd.read_csv("results_extraction_comparison.csv")

results.to_excel("results_extraction_comparison.xlsx", index=False)